In [24]:
from sage.misc.misc_c import prod

from lib.Hypermatrix_Algebra_tst import HM, var_list

import numpy

In [25]:
sz = 2; HV = HM(*(sz for i in range(sz)), 'v')

In [26]:
HV.printHM() # only valid for sz < 4 # fixed!
# print(V)

[:, :]=
[v00 v01]
[v10 v11]


In [27]:
def lexSymbol(digits, sz, terms, f):
    raw = int(str(f)[1:]) # will only work when sz <= 10 # also fixed!
    return sum((raw // 10**(digits*(terms-i-1))) % 10**digits * sz**i for i in range(terms))


lexSymbol(1, sz, sz, HV[0,0])

0

In [28]:
HE = HM(*(sz**sz for i in range(3)), 'e')

HE.printHM()

[:, :, 0]=
[e000 e010 e020 e030]
[e100 e110 e120 e130]
[e200 e210 e220 e230]
[e300 e310 e320 e330]

[:, :, 1]=
[e001 e011 e021 e031]
[e101 e111 e121 e131]
[e201 e211 e221 e231]
[e301 e311 e321 e331]

[:, :, 2]=
[e002 e012 e022 e032]
[e102 e112 e122 e132]
[e202 e212 e222 e232]
[e302 e312 e322 e332]

[:, :, 3]=
[e003 e013 e023 e033]
[e103 e113 e123 e133]
[e203 e213 e223 e233]
[e303 e313 e323 e333]



In [29]:
lexSymbol(2, sz**sz, sz, HE[0, 1, 0])

40

In [30]:
E = HM(*(sz for i in range(2)), 'a')
E.printHM()

[:, :]=
[a00 a01]
[a10 a11]


In [31]:
Lx = var_list('x', sz**sz); Ly = var_list('y', sz**sz); Lz = var_list('z', sz**sz)

In [32]:


def listFromSymbol(digits, terms, f):
    raw = int(str(f)[1:])
    return [(raw // 10**(digits*(terms-i-1))) % 10**digits for i in range(terms)]



def composeSymbols(HV, digits, terms, f1, f2):
    
    f1list = listFromSymbol(digits, terms, f1)
    f2list = listFromSymbol(digits, terms, f2)
    
    return HV[tuple(f1list[f2list[i]] for i in range(sz))]

print(composeSymbols(HV, 1, sz, HV[1,1], HV[0,1])) # HV[1,1,0], HV[0,1,0]))
print(composeSymbols(HV, 1, sz, HV[0,1], HV[1,1])) # HV[0,1,0], HV[1,1,0]))

def lexList(sz, f):
    return sum((d * sz**i for i, d in enumerate(f)))

def composeLists(terms, f1, f2):
    return [f1[f2[i]] for i in range(terms)]

print(lexList(3, composeLists(3, [1, 1, 0], [0, 1, 0])))

def listFromLex(sz, terms, lex):
    return [(lex // (sz ** i)) % sz for i in range(terms)]

def SymbolFromLex(HV, sz, terms, lex):
    return HV[tuple((lex // (sz ** i)) % sz for i in range(terms))]

def composeLex(sz, terms, f1, f2):
    f1list = listFromLex(sz, terms, f1)
    f2list = listFromLex(sz, terms, f2)
    
    return lexList(sz, (f1list[f2list[i]] for i in range(sz)))

print(SymbolFromLex(HV, sz, sz, lexList(3, composeLists(3, [1, 1, 0], [0, 1, 0]))))

def SymbolFromList(f):
    return HV[tuple(f)]

print(SymbolFromList([1, 1]))

def get_ident_func(sz):
    return [i for i in range(sz)], lexList(sz, (i for i in range(sz)))

v11
v11
13
v10
v11


In [33]:
monoids = prod(1 + HE[x, y, composeLex(sz, sz, x, y)] * Lx[x] * Ly[y] * Lz[composeLex(sz, sz, x, y)] for x in range(sz**sz) for y in range(sz**sz))

_, ident = get_ident_func(2)

# print(monoids.expand().diff(Lx[ident]).diff(Ly[ident]).diff(Lz[ident]))

# expand = monoids.expand()

In [34]:
with_ident = monoids.diff(Lx[ident]).diff(Ly[ident]).diff(Lz[ident]) # expand.diff(Lx[ident]).diff(Ly[ident]).diff(Lz[ident])

In [35]:
temp = 1
for i in range(sz**sz):
     temp2 = with_ident.diff(Lx[i]).diff(Ly[i])
     temp += temp2.diff(Lz[ident]) + temp2.diff(Lz[i])

In [39]:
eval_temp = temp({Lx[0]:0,Lx[1]:0,Lx[2]:0,Lx[3]:0,Ly[0]:0,Ly[1]:0,Ly[2]:0,Ly[3]:0,Lz[0]:0,Lz[1]:0,Lz[2]:0,Lz[3]:0})

In [40]:
print(eval_temp)

e000*e222 + 2*e112*e222 + e222*e333 + 1
